<a href="https://colab.research.google.com/github/cafechungkhoan/chu_gia/blob/master/Auto_ML_Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycaret

In [2]:
from google.colab import files
uploaded = files.upload()

Saving anomaly detection vnindex.xls to anomaly detection vnindex.xls


In [3]:
import pandas as pd
import io
data = pd.read_excel(io.BytesIO(uploaded['anomaly detection vnindex.xls']))
print(data)

*** No CODEPAGE record, no encoding_override: will use 'ascii'
            Date      Time        Close
0     2021/07/08  09:00:00  1374.680054
1     2021/07/07  09:00:00  1388.550000
2     2021/07/06  09:00:00  1354.790000
3     2021/07/05  09:00:00  1411.130000
4     2021/07/02  09:00:00  1420.270000
...          ...       ...          ...
2997  2009/07/07  09:00:00   447.630000
2998  2009/07/06  09:00:00   454.270000
2999  2009/07/03  09:00:00   435.440000
3000  2009/07/02  09:00:00   433.790000
3001  2009/07/01  09:00:00   430.000000

[3002 rows x 3 columns]


In [4]:
data['Date'] = pd.to_datetime(data['Date'])
data = data.drop(['Time'],axis = 1)
data = data.dropna()
data

,Date,Close
0,2021-07-08,1374.680054
1,2021-07-07,1388.550000
2,2021-07-06,1354.790000
3,2021-07-05,1411.130000
4,2021-07-02,1420.270000
...,...,...
2997,2009-07-07,447.630000
2998,2009-07-06,454.270000
2999,2009-07-03,435.440000
3000,2009-07-02,433.790000


In [7]:
import numpy as np
# extract month and year from dates
data['Month'] = [i.month for i in data['Date']]
data['Year'] = [i.year for i in data['Date']]
# create a sequence of numbers
data['Series'] = np.arange(1,len(data)+1)
# drop unnecessary columns and re-arrange
data.drop(['Date'], axis=1, inplace=True)
data = data[['Series', 'Year', 'Month', 'Close']] 
# check the head of the dataset
data.head()

,Series,Year,Month,Close
0,1,2021,7,1374.680054
1,2,2021,7,1388.550000
2,3,2021,7,1354.790000
3,4,2021,7,1411.130000
4,5,2021,7,1420.270000


In [9]:
# split data into train-test set
train = data[data['Year'] < 2020]
test = data[data['Year'] >= 2020]
# check shape
train.shape, test.shape

((2624, 4), (378, 4))

In [ ]:
# import the regression module
from pycaret.regression import *
# initialize setup
s = setup(data = train, test_data = test, target = 'Close', fold_strategy = 'timeseries', numeric_features = ['Year', 'Series'], fold = 3, transform_target = True, session_id = 123)

In [12]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,107.7934,16715.4831,118.7870,-4.3746,0.2222,0.1961,0.0233
lightgbm,Light Gradient Boosting Machine,112.2195,19887.8225,130.3030,-5.0066,0.2248,0.2129,0.2100
dt,Decision Tree Regressor,113.9584,20409.3056,131.4294,-5.1979,0.2265,0.2150,0.0233
rf,Random Forest Regressor,114.2505,20478.6020,131.7657,-5.2079,0.2271,0.2159,0.4667
ada,AdaBoost Regressor,116.7446,21061.2216,132.1314,-5.3554,0.2255,0.2211,0.0600
gbr,Gradient Boosting Regressor,124.9498,24765.8176,141.1001,-6.5767,0.2381,0.2338,0.0800
et,Extra Trees Regressor,125.1249,25289.2309,143.0973,-6.7323,0.2411,0.2344,0.3967
br,Bayesian Ridge,126.5189,24403.1653,139.5713,-6.8237,0.2495,0.2282,0.0267
ridge,Ridge Regression,126.8562,24573.5805,140.0090,-6.8777,0.2501,0.2288,0.0233
lar,Least Angle Regression,127.1401,24729.9127,140.3848,-6.9275,0.2505,0.2293,0.0267


In [13]:
prediction_holdout = predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Orthogonal Matching Pursuit,136.8618,26392.4012,162.4574,0.2436,0.1713,0.1538


In [16]:
final_best = finalize_model(best)

In [17]:
future_dates = pd.date_range(start = '2009-07-01', end = '2021-07-08', freq = 'MS')
future_df = pd.DataFrame()
future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates]    
future_df['Series'] = np.arange(145,(145+len(future_dates)))
future_df.head()

,Month,Year,Series
0,7,2009,145
1,8,2009,146
2,9,2009,147
3,10,2009,148
4,11,2009,149


In [25]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future.tail(10)

,Month,Year,Series,Label
135,10,2020,280,1054.693900
136,11,2020,281,1054.693900
137,12,2020,282,1054.693900
138,1,2021,283,1225.475906
139,2,2021,284,1225.475906
140,3,2021,285,1225.475906
141,4,2021,286,1225.475906
142,5,2021,287,1225.475906
143,6,2021,288,1225.475906
144,7,2021,289,1225.475906
